In [29]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import TextVectorization, Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import numpy as np

In [30]:
# Dataset (contoh data)
data = {
    'Penyakit': ['Batuk', 'Nyeri Otot', 'Pusing', 'Sakit Perut', 'Pilek'],
    'Jamu': ['Jahe Hangat', 'Kunyit Asam', 'Wedang Uwuh', 'Jamu Kencur', 'Teh Herbal']
}

In [40]:
# # Encode label jamu ke dalam bentuk numerik
# jamu_to_index = {jamu: i for i, jamu in enumerate(set(data['Jamu']))}
# index_to_jamu = {i: jamu for jamu, i in jamu_to_index.items()}
# y = np.array([jamu_to_index[jamu] for jamu in data['Jamu']])

# Membuat kamus untuk mengonversi nama jamu menjadi indeks numerik
jamu_to_index = {}
index_to_jamu = {}

# Mengisi kamus dengan perulangan
for i, jamu in enumerate(set(data['Jamu'])):
    jamu_to_index[jamu] = i
    index_to_jamu[i] = jamu

# Mengubah data 'Jamu' menjadi array numerik berdasarkan kamus jamu_to_index
y = [jamu_to_index[jamu] for jamu in data['Jamu']]

print("Encoded Labels:", y)
print("Jamu to Index:", jamu_to_index)
print("Index to Jamu:", index_to_jamu)


Encoded Labels: [1, 3, 2, 0, 4]
Jamu to Index: {'Jamu Kencur': 0, 'Jahe Hangat': 1, 'Wedang Uwuh': 2, 'Kunyit Asam': 3, 'Teh Herbal': 4}
Index to Jamu: {0: 'Jamu Kencur', 1: 'Jahe Hangat', 2: 'Wedang Uwuh', 3: 'Kunyit Asam', 4: 'Teh Herbal'}


In [41]:
# Bagi dataset menjadi data training dan testing
X_train, X_test, y_train, y_test = train_test_split(data['Penyakit'], y, test_size=0.2, random_state=42)


In [42]:
# Konfigurasi TextVectorization untuk preprocessing
vectorizer = TextVectorization(output_mode='int', max_tokens=1000)
vectorizer.adapt(X_train)


In [43]:
# Membangun Model TensorFlow
model = Sequential([
    vectorizer,
    Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=16, mask_zero=True),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(len(jamu_to_index), activation='softmax')
])

In [44]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [45]:
# Ubah X_train dan X_test menjadi tensor yang telah di-vectorize
X_train = vectorizer(X_train)
X_test = vectorizer(X_test)

In [46]:
# Training model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


ValueError: Unrecognized data type: x=[[4 0]
 [3 0]
 [6 0]
 [2 5]] (of type <class 'tensorflow.python.framework.ops.EagerTensor'>)

In [ ]:
# Fungsi untuk memberikan rekomendasi
def rekomendasi_jamu(input_penyakit):
    input_vector = tf.constant([input_penyakit])
    pred = model.predict(input_vector)
    jamu_index = np.argmax(pred[0])
    return index_to_jamu[jamu_index]


In [ ]:
# Contoh penggunaan
penyakit_input = 'Batuk'
print(f"Penyakit: {penyakit_input}, Rekomendasi Jamu: {rekomendasi_jamu(penyakit_input)}")